In [30]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

# Function to simulate price
def simulate_price(S0, mu, sigma, days):
    dt = 1 / 252  # Daily time step (assuming 252 trading days in a year)
    price = [S0]
    for _ in range(days - 1):
        random_shock = np.random.normal(loc=mu * dt, scale=sigma * np.sqrt(dt))
        price.append(price[-1] * np.exp(random_shock))
    return np.array(price)

# Set random seed for reproducibility
np.random.seed(42)

# Parameters for simulation
num_days = 90  # Total days for simulation
days_per_contract = num_days // 3
dates = pd.date_range(start="2021-01-01", periods=num_days, freq="D")

# Simulate prices for three futures contracts
price_near = simulate_price(100, 0.05, 0.2, days_per_contract)
price_mid = simulate_price(105, 0.04, 0.2, days_per_contract)
price_far = simulate_price(110, 0.03, 0.2, days_per_contract)

# Combine simulated prices into a single dataset
prices = np.concatenate([price_near, price_mid, price_far])
contracts = ["Near"] * days_per_contract + ["Mid"] * days_per_contract + ["Far"] * days_per_contract
data = {"Date": dates, "Contract": contracts, "Price": prices}
df = pd.DataFrame(data)

# Define roll dates (last day of each contract)
roll_dates = [dates[days_per_contract - 1], dates[2 * days_per_contract - 1]]

# Continuous price series initialization
df["Adjusted_Price"] = df["Price"]
df["Cumulative_Factor"] = 1.0  # Initialize cumulative adjustment factor
adjustment_factor = 1.0  # Start with no adjustment

# Apply multiplicative forward adjustments and track cumulative factors
for roll_date in roll_dates:
    roll_index = df[df["Date"] == roll_date].index[0]
    prev_price = df.loc[roll_index, "Price"]  # Expiring contract price
    new_price = df.loc[roll_index + 1, "Price"]  # Incoming contract price
    factor = prev_price / new_price  # Adjustment factor for this roll
    adjustment_factor *= factor  # Update cumulative adjustment factor
    df.loc[roll_index + 1 :, "Adjusted_Price"] *= factor
    df.loc[roll_index + 1 :, "Cumulative_Factor"] = adjustment_factor

# Back out the original prices using cumulative factor
df["Backed_Price"] = df["Adjusted_Price"] / df["Cumulative_Factor"]

# Plot using Plotly
fig = go.Figure()

fig.add_trace(go.Scatter(x=df["Date"], y=df["Price"], mode="lines+markers", name="Original Prices"))
fig.add_trace(go.Scatter(x=df["Date"], y=df["Adjusted_Price"], mode="lines+markers", name="Adjusted Prices (Continuous)"))
fig.add_trace(go.Scatter(x=df["Date"], y=df["Backed_Price"], mode="lines+markers", name="Backed Prices"))

# Add vertical lines for roll dates
for roll_date in roll_dates:
    fig.add_vline(x=roll_date, line=dict(color="grey", dash="dash"), name="Roll Date")

fig.update_layout(
    title="Continuous Futures Price Series with Forward Adjustments",
    xaxis_title="Date",
    yaxis_title="Price",
    legend_title="Legend",
    template="plotly_white"
)

fig.show()
